# Import & Load Election results data

### Config

In [1]:
import sys
import configparser
config = configparser.ConfigParser()
config.read_file(open('../../../settings.ini'))

sys.path.insert(0, config.get('PATHS','libs_path'))
engine_path = config.get('DATABASE','engine_path')

### Main params

In [2]:
remote_path = 'http://pacha.datawheel.us/votaciones/'
local_path = '../data/'

### Imports

In [3]:
import postgres #from local file postgres.py
from commons import inline_table_xml, download_file #from local file commons.py

import json
import pandas as pd
from sqlalchemy import create_engine

### Load file

In [4]:
p1 = download_file(remote_path+'02_presidenciales/',local_path,'archivo_limpio_primera_vuelta.csv')
p2 = download_file(remote_path+'02_presidenciales/',local_path,'archivo_limpio_segunda_vuelta.csv')
sn = download_file(remote_path+'01_parlamentarias/',local_path,'archivo_limpio_senadores.csv')
mn = download_file(remote_path+'03_municipales/',local_path,'archivo_limpio_municipales.csv')
dp = download_file(remote_path+'01_parlamentarias/',local_path,'archivo_limpio_diputados.csv')

frames = [p1, p2, sn, dp, mn]

df = pd.concat(frames)

df = df[['year','election_type','comuna_datachile_id','partido_id','mesa_id','candidato','votos']]
df = df.rename(columns={'votos':'votes','candidato':'candidate','partido_id':'party_id','mesa_id':'sex_id','election_type':'election_id'})
df = df.astype({'comuna_datachile_id':'int', 'votes':'int','party_id':'int','sex_id':'int','election_id':'int','year':'int'})

list(df)

Downloading... http://pacha.datawheel.us/votaciones/02_presidenciales/archivo_limpio_primera_vuelta.csv
Downloading... http://pacha.datawheel.us/votaciones/02_presidenciales/archivo_limpio_segunda_vuelta.csv
Downloading... http://pacha.datawheel.us/votaciones/01_parlamentarias/archivo_limpio_senadores.csv
Downloading... http://pacha.datawheel.us/votaciones/03_municipales/archivo_limpio_municipales.csv
Downloading... http://pacha.datawheel.us/votaciones/01_parlamentarias/archivo_limpio_diputados.csv


['year',
 'election_id',
 'comuna_datachile_id',
 'party_id',
 'sex_id',
 'candidate',
 'votes']

### Ingest

In [5]:
engine = create_engine(engine_path)
db = postgres.PostgresDriver(engine)
db.to_sql(df, 'politics', 'fact_election_results')

DROP TABLE IF EXISTS politics.fact_election_results;
CREATE TABLE "politics"."fact_election_results" (
"year" INTEGER,
  "election_id" INTEGER,
  "comuna_datachile_id" INTEGER,
  "party_id" INTEGER,
  "sex_id" INTEGER,
  "candidate" TEXT,
  "votes" INTEGER
)
COPY "politics"."fact_election_results" ("year","election_id","comuna_datachile_id","party_id","sex_id","candidate","votes") FROM STDIN WITH CSV HEADER DELIMITER ',';


### Indexes & FK

In [6]:
engine.execute("""
ALTER TABLE politics.fact_election_results
  ADD COLUMN date_id INTEGER; 
""")

engine.execute("""
UPDATE politics.fact_election_results
SET date_id = dim_date.id
FROM public.dim_date
WHERE dim_date.the_year = politics.fact_election_results.year
      AND dim_date.month_of_year = 1
      AND dim_date.day_of_month = 1
""")

engine.execute("""
CREATE INDEX fact_election_results_comuna_datachile_id 
ON politics.fact_election_results (comuna_datachile_id)
""")

### Related dim

```xml
<InlineTable alias="sex">
  <ColumnDefs>
    <ColumnDef name="id" type="Numeric" />
    <ColumnDef name="description" type="String" />
    <ColumnDef name="es_description" type="String" />
  </ColumnDefs>
  <Rows>
    <Row>
      <Value column="id">0</Value>
      <Value column="description">No informa</Value>
      <Value column="es_description">No informa</Value>
    </Row>
  <Row>
      <Value column="id">1</Value>
      <Value column="description">Women</Value>
      <Value column="es_description">Mujer</Value>
    </Row>
  <Row>
      <Value column="id">2</Value>
      <Value column="description">Men</Value>
      <Value column="es_description">Hombre</Value>
    </Row>
  </Rows>
</InlineTable>
```

In [7]:
d1 = download_file(remote_path+'01_parlamentarias/',local_path,'election_id.csv')
print (inline_table_xml(d1, 'election', 'election_id', 'election'))

Downloading... http://pacha.datawheel.us/votaciones/01_parlamentarias/election_id.csv

<InlineTable alias="election">
  <ColumnDefs>
    <ColumnDef name="id" type="Numeric" />
    <ColumnDef name="description" type="String" />
    <ColumnDef name="es_description" type="String" />
  </ColumnDefs>
  <Rows>
    <Row>
      <Value column="id">1</Value>
      <Value column="description">Presidente - Primera vuelta</Value>
      <Value column="es_description">Presidente - Primera vuelta</Value>
    </Row>
  <Row>
      <Value column="id">2</Value>
      <Value column="description">Presidente - Segunda vuelta</Value>
      <Value column="es_description">Presidente - Segunda vuelta</Value>
    </Row>
  <Row>
      <Value column="id">3</Value>
      <Value column="description">Senador</Value>
      <Value column="es_description">Senador</Value>
    </Row>
  <Row>
      <Value column="id">4</Value>
      <Value column="description">Diputado</Value>
      <Value column="es_description">Diputado</

In [8]:
d2 = download_file(remote_path+'01_parlamentarias/',local_path,'partidos_id.csv')
print (inline_table_xml(d2, 'partido', 'partido_id', 'partido'))

Downloading... http://pacha.datawheel.us/votaciones/01_parlamentarias/partidos_id.csv

<InlineTable alias="partido">
  <ColumnDefs>
    <ColumnDef name="id" type="Numeric" />
    <ColumnDef name="description" type="String" />
    <ColumnDef name="es_description" type="String" />
  </ColumnDefs>
  <Rows>
    <Row>
      <Value column="id">1</Value>
      <Value column="description">Partido Comunista de Chile (PCCh)</Value>
      <Value column="es_description">Partido Comunista de Chile (PCCh)</Value>
    </Row>
  <Row>
      <Value column="id">2</Value>
      <Value column="description">Partido Radical Socialdemocrata (PRSD)</Value>
      <Value column="es_description">Partido Radical Socialdemocrata (PRSD)</Value>
    </Row>
  <Row>
      <Value column="id">3</Value>
      <Value column="description">Partido Regionalista de los Independientes (PRI)</Value>
      <Value column="es_description">Partido Regionalista de los Independientes (PRI)</Value>
    </Row>
  <Row>
      <Value colum